In [11]:
import pandas as pd
import numpy as np
import pandahouse as ph

# Финальный проект. Вариант 2.
---

## Задание 2. SQL

### 2.1. Очень усердные ученики.

#### 2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

#### 2.1.2 Задача

Дана таблица default.peas.

- st_id	- ID ученика
- timest - Время решения карточки
- correct -	Правильно ли решена горошина?
- subject - Дисциплина, в которой находится горошина

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов. 
 NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

---
Для изложения решения и возможности его быстрой проверки используется библиотека `pandahouse`.
Запросы сначала составлялись и проверялись в `Tabix`, затем здесь.


In [12]:
# создадим словарь, содержащий информацию для подключения к БД:

connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [13]:
# создаем запрос, db автоматически заменится на значение из database
q1 = '''
    WITH 
    (SELECT toMonth(MAX(timest)) FROM default.peas) 
AS
    last_month 

SELECT 
    st_id,
    correct_peas
FROM (
    SELECT 
        st_id,
        SUM(correct) as correct_peas
    FROM 
        default.peas
    WHERE 
        toMonth(timest) = last_month
    GROUP BY 
        st_id
    )
WHERE correct_peas >= 20
ORDER BY correct_peas DESC
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
q1_df = ph.read_clickhouse(query=q1, connection=connection_default)

In [14]:
# Результат работы запроса:
q1_df

,st_id,correct_peas
0,700123,126
1,976310,121
2,577659,111
3,234705,81
4,434775,79
...,...,...
131,544247,20
132,147316,20
133,269738,20
134,348454,20


## 2.2 Оптимизация воронки

### 2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

### 2.2.2 Задача

Дана таблицы: default.peas (см. выше), default.studs и default.final_project_check.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [18]:
q2 = '''
WITH base_table  AS (
    SELECT 
        p.st_id as st_id, 
        p.correct as correct, 
        p.subject as subject, 
        s.test_grp as grp,
        c.money as money
    FROM 
        (SELECT toInt64(st_id) as st_id, subject, SUM(correct) as correct
        FROM {db}.peas
        GROUP BY st_id, subject)
        as p
    LEFT JOIN 
        {db}.studs 
        as s
    ON p.st_id = toInt64(s.st_id)
    LEFT JOIN 
        (SELECT st_id, subject, sum(money) as money
        FROM {db}.final_project_check
        GROUP BY st_id, subject) 
        as c 
    ON p.st_id = toInt64(c.st_id) AND p.subject = c.subject
    ),
active_and_buyers AS (
    SELECT 
        st_id, 
        IF(SUM(correct) > 10, 1, 0) as is_active,
        IF(SUM(money) > 0, 1, 0) as is_buy
    FROM base_table
    GROUP BY st_id),
math_users AS (
    SELECT 
        st_id, 
        IF(SUM(correct) >= 2, 1, 0) as is_active_m,
        IF(SUM(money) > 0, 1, 0) as is_buy_m
    FROM base_table
    WHERE subject = 'Math'
    GROUP BY st_id)

SELECT
    grp,
    SUM(money) / COUNT(DISTINCT st_id) as arpu,
    SUM(money) / SUM(is_active) as arpau,
    ROUND(100 * SUM(is_buy)/COUNT(DISTINCT st_id), 2) as cr,
    ROUND(100 * SUM(is_buy)/SUM(is_active), 2) as cr_active,
    ROUND(100 * SUM(is_buy_m)/SUM(is_active_m), 2) as cr_math
FROM ( 
    SELECT
        bt.st_id as st_id,
        bt.grp as grp,
        bt.money as money,
        ab.is_active as is_active,
        ab.is_buy as is_buy,
        mu.is_active_m as is_active_m,
        mu.is_buy_m as is_buy_m
    FROM 
        (SELECT  
            st_id, 
            grp,
            SUM(correct) as correct,
            SUM(money) as money
        FROM 
            base_table
        GROUP BY st_id, grp
        ) as bt
    LEFT JOIN 
        active_and_buyers as ab
    ON bt.st_id = ab.st_id
    LEFT JOIN
        math_users as mu
    ON bt.st_id = mu.st_id
    )
GROUP BY grp
'''

In [19]:
q2_df = ph.read_clickhouse(query=q2, connection=connection_default)

# Результат работы запроса:
q2_df

,grp,arpu,arpau,cr,cr_active,cr_math
0,control,4121.212121,5354.330709,4.85,6.3,6.12
1,pilot,7902.097902,11770.833333,8.39,12.5,9.52
